In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [60]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor, AdaBoostRegressor
import xgboost

In [61]:
df = pd.read_csv("../data/heat.csv")
df = df.copy()

# Lenght imputation

In [62]:
df_length = df.copy()
# df_dh.drop(columns="id", inplace=True)

### Target

In [63]:
target = df_length[df_length["length [mm]"].isnull()]
target = target[(target["pressure [MPa]"].notna())]
target.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4119 entries, 16 to 31640
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               4119 non-null   int64  
 1   length [mm]      0 non-null      float64
 2   pressure [MPa]   4119 non-null   float64
 3   chf_exp [MW/m2]  4119 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 160.9 KB


### Training set

In [64]:
df_length.dropna(how="any", inplace=True)
df_length.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23073 entries, 0 to 31643
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               23073 non-null  int64  
 1   length [mm]      23073 non-null  float64
 2   pressure [MPa]   23073 non-null  float64
 3   chf_exp [MW/m2]  23073 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 901.3 KB


In [65]:
X = df_length[["pressure [MPa]", "chf_exp [MW/m2]"]]
y = df_length["length [mm]"]

In [66]:
X.describe()

,pressure [MPa],chf_exp [MW/m2]
count,23073.000000,23073.000000
mean,10.633725,3.805145
std,4.345228,1.974666
min,0.100000,0.800000
25%,6.890000,2.400000
50%,11.070000,3.400000
75%,13.790000,4.700000
max,20.680000,19.300000


In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [68]:
rand_forest = Pipeline(steps=[
                          ("scaler",StandardScaler()),
                          ("rad_forest",RandomForestRegressor(random_state=42))
                         ])

svm = Pipeline(steps=[("scaler",StandardScaler()),
                          ("SVR",SVR())
                         ])

ada_reg = Pipeline(steps=[("scaler",StandardScaler()),
                          ("ada_reg",AdaBoostRegressor(n_estimators=200,
                            random_state=42))
                         ])

gbr_reg = Pipeline(steps=[("scaler",StandardScaler()),
                          ("gbr_reg",GradientBoostingRegressor(random_state=42))
                         ])

vot_reg = Pipeline(steps=[("scaler",StandardScaler()),
                          ("vot_reg",VotingRegressor(estimators=200))
                         ])


# Definimos sus hiperparametros

rand_forest_param = {
    'rad_forest__n_estimators': [10, 100, 1000],
    'rad_forest__min_samples_leaf': [2, 4],
    'rad_forest__max_features': [1, 2]
    }

svm_param = {
    'SVR__C': [0.1, 1, 10],
    'SVR__kernel': ["linear", "rbf", "poly"],
    'SVR__gamma': ["scale", "auto"]
}

ada_param = {
    'ada_reg__n_estimators': [100, 200, 300],
    'ada_reg__learning_rate': [0.01, 0.1, 1]
}

gbr_param = {
    'gbr_reg__n_estimators': [100, 200, 300],
    'gbr_reg__learning_rate': [0.01, 0.1, 1],
    'gbr_reg__max_depth': [3, 5, 7]
}


# GridSearch

gs_rand_forest = GridSearchCV(rand_forest,
                            rand_forest_param,
                            cv=10,
                            scoring=["neg_mean_squared_error", "r2"],
                            verbose=1,
                            n_jobs=-1, refit="neg_mean_squared_error")

gs_svm = GridSearchCV(svm,
                        svm_param,
                        cv=10,
                        scoring=["neg_mean_squared_error", "r2"],
                        verbose=1,
                        n_jobs=-1, refit="neg_mean_squared_error")

gs_ada = GridSearchCV(ada_reg,
                        ada_param,
                        cv=10,
                        scoring=["neg_mean_squared_error", "r2"],
                        verbose=1,
                        n_jobs=-1, refit="neg_mean_squared_error")

gs_gbr = GridSearchCV(gbr_reg,
                        gbr_param,
                        cv=10,
                        scoring=["neg_mean_squared_error", "r2"],
                        verbose=1,
                        n_jobs=-1, refit="neg_mean_squared_error")

grids = {"gs_rand_forest":gs_rand_forest,
         "gs_svm":gs_svm,
         "gs_ada":gs_ada,
         "gs_gbr":gs_gbr,
         }

In [69]:
%%time

for nombre, grid_search in grids.items():
    grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 18 candidates, totalling 180 fits
Fitting 10 folds for each of 9 candidates, totalling 90 fits
Fitting 10 folds for each of 27 candidates, totalling 270 fits
CPU times: total: 14.5 s
Wall time: 5min 3s


In [70]:
best_grids = [(i, j.best_score_) for i, j in grids.items()]

best_grids = pd.DataFrame(best_grids, columns=["Grid", "Best score"]).sort_values(by="Best score", ascending=False)
best_grids

,Grid,Best score
3,gs_gbr,-227116.831055
0,gs_rand_forest,-231605.601698
2,gs_ada,-307750.598345
1,gs_svm,-352410.989033


In [71]:
# El mejor modelo ha sido
best_model = grids["gs_gbr"]
mejor_modelo = best_model.best_estimator_
mejor_modelo.fit(X_train, y_train)
mejor_modelo.score(X_test, y_test)

0.4939393512692367

In [72]:
tree_reg = DecisionTreeRegressor()

lasso = Pipeline(steps=[("scaler",StandardScaler()),
                          ("lasso",Lasso())
                         ])


tree_reg_param = {
    'max_depth': [3, 5, 7],
    'min_samples_leaf': [2, 4],
    'max_features': [1, 2]
    }

lasso_param = {
    'lasso__alpha': [0.1, 1.0, 10.0],
    'lasso__tol': [0.001, 0.0001, 0.00001],
    }

gs_tree = GridSearchCV(tree_reg,
                        tree_reg_param,
                        cv=10,
                        scoring=["neg_mean_squared_error", "r2"],
                        verbose=1,
                        n_jobs=-1, refit="neg_mean_squared_error")

gs_lasso = GridSearchCV(lasso,
                        lasso_param,
                        cv=10,
                        scoring=["neg_mean_squared_error", "r2"],
                        verbose=1,
                        n_jobs=-1, refit="neg_mean_squared_error")

grids_2 = {"gs_tree_reg":gs_tree,
         "gs_lasso":gs_lasso,
         }

In [73]:
%%time

for nombre, grid_search in grids_2.items():
    grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 9 candidates, totalling 90 fits
CPU times: total: 15.6 ms
Wall time: 288 ms


In [74]:
best_grids = [(i, j.best_score_) for i, j in grids_2.items()]

best_grids = pd.DataFrame(best_grids, columns=["Grid", "Best score"]).sort_values(by="Best score", ascending=False)
best_grids

,Grid,Best score
0,gs_tree_reg,-245409.381489
1,gs_lasso,-402021.254260


In [75]:
# El mejor modelo ha sido
best_model = grids_2["gs_tree_reg"]
mejor_modelo = best_model.best_estimator_
mejor_modelo.fit(X_train, y_train)
mejor_modelo.score(X_test, y_test)

0.45126365666956236

In [76]:
best_model = pd.DataFrame(gs_tree.cv_results_)
model_ranking = best_model[["params", "rank_test_r2", "rank_test_neg_mean_squared_error"]].sort_values(by="rank_test_r2")

model_ranking.loc[11, "params"]

{'max_depth': 7, 'max_features': 2, 'min_samples_leaf': 4}

### Ningún modelo lo hace bien así que imputamos por media

In [81]:
df_length = df.copy()

final_target = df_length

final_target["length [mm]"] =  final_target["length [mm]"].fillna(final_target["length [mm]"].mean())

# Pressure imputation

In [22]:
df_pressure = df.copy()

In [26]:
target_2 = df_pressure[df_pressure["pressure [MPa]"].isnull()]
target_2 = target_2[(target_2["length [mm]"].notna())]
target_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3812 entries, 1 to 31639
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               3812 non-null   int64  
 1   length [mm]      3812 non-null   float64
 2   pressure [MPa]   0 non-null      float64
 3   chf_exp [MW/m2]  3812 non-null   float64
dtypes: float64(3), int64(1)
memory usage: 148.9 KB


In [30]:
df_pressure.dropna(how="any", inplace=True)
X = df_pressure[["length [mm]", "chf_exp [MW/m2]"]]
y = df_pressure["pressure [MPa]"]
X.describe()

,length [mm],chf_exp [MW/m2]
count,23073.000000,23073.000000
mean,831.467429,3.805145
std,670.306167,1.974666
min,10.000000,0.800000
25%,318.000000,2.400000
50%,610.000000,3.400000
75%,914.000000,4.700000
max,3048.000000,19.300000


In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

In [32]:
rand_forest = Pipeline(steps=[
                          ("scaler",StandardScaler()),
                          ("rad_forest",RandomForestRegressor(random_state=42))
                         ])

svm = Pipeline(steps=[("scaler",StandardScaler()),
                          ("SVR",SVR())
                         ])

ada_reg = Pipeline(steps=[("scaler",StandardScaler()),
                          ("ada_reg",AdaBoostRegressor(n_estimators=200,
                            random_state=42))
                         ])

gbr_reg = Pipeline(steps=[("scaler",StandardScaler()),
                          ("gbr_reg",GradientBoostingRegressor(random_state=42))
                         ])

vot_reg = Pipeline(steps=[("scaler",StandardScaler()),
                          ("vot_reg",VotingRegressor(estimators=200))
                         ])

# Definimos sus hiperparametros

rand_forest_param = {
    'rad_forest__n_estimators': [10, 100, 1000],
    'rad_forest__min_samples_leaf': [2, 4],
    'rad_forest__max_features': [1, 2]
    }

svm_param = {
    'SVR__C': [0.1, 1, 10],
    'SVR__kernel': ["linear", "rbf", "poly"],
    'SVR__gamma': ["scale", "auto"]
}

ada_param = {
    'ada_reg__n_estimators': [100, 200, 300],
    'ada_reg__learning_rate': [0.01, 0.1, 1]
}

gbr_param = {
    'gbr_reg__n_estimators': [100, 200, 300],
    'gbr_reg__learning_rate': [0.01, 0.1, 1],
    'gbr_reg__max_depth': [3, 5, 7]
}


# GridSearch

gs_rand_forest = GridSearchCV(rand_forest,
                            rand_forest_param,
                            cv=10,
                            scoring=["neg_mean_squared_error", "r2"],
                            verbose=1,
                            n_jobs=-1, refit="neg_mean_squared_error")

gs_svm = GridSearchCV(svm,
                        svm_param,
                        cv=10,
                        scoring=["neg_mean_squared_error", "r2"],
                        verbose=1,
                        n_jobs=-1, refit="neg_mean_squared_error")

gs_ada = GridSearchCV(ada_reg,
                        ada_param,
                        cv=10,
                        scoring=["neg_mean_squared_error", "r2"],
                        verbose=1,
                        n_jobs=-1, refit="neg_mean_squared_error")

gs_gbr = GridSearchCV(gbr_reg,
                        gbr_param,
                        cv=10,
                        scoring=["neg_mean_squared_error", "r2"],
                        verbose=1,
                        n_jobs=-1, refit="neg_mean_squared_error")

grids = {"gs_rand_forest":gs_rand_forest,
         "gs_svm":gs_svm,
         "gs_ada":gs_ada,
         "gs_gbr":gs_gbr,
         }

In [33]:
%%time

for nombre, grid_search in grids.items():
    grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
Fitting 10 folds for each of 18 candidates, totalling 180 fits
Fitting 10 folds for each of 9 candidates, totalling 90 fits
Fitting 10 folds for each of 27 candidates, totalling 270 fits
CPU times: total: 10.9 s
Wall time: 5min 45s


In [34]:
best_grids = [(i, j.best_score_) for i, j in grids.items()]

best_grids = pd.DataFrame(best_grids, columns=["Grid", "Best score"]).sort_values(by="Best score", ascending=False)
best_grids

,Grid,Best score
3,gs_gbr,-4.618646
0,gs_rand_forest,-4.770583
2,gs_ada,-11.593515
1,gs_svm,-13.575802


In [35]:
# El mejor modelo ha sido
best_model = grids["gs_gbr"]
mejor_modelo = best_model.best_estimator_
mejor_modelo.fit(X_train, y_train)
mejor_modelo.score(X_test, y_test)

0.762115621377701

In [37]:
best_model = pd.DataFrame(gs_gbr.cv_results_)
model_ranking = best_model[["params", "rank_test_r2", "rank_test_neg_mean_squared_error"]].sort_values(by="rank_test_r2")

model_ranking.loc[11, "params"]

{'gbr_reg__learning_rate': 0.1,
 'gbr_reg__max_depth': 3,
 'gbr_reg__n_estimators': 300}

In [38]:
# Elegir modelo

gb_reg = GradientBoostingRegressor(learning_rate=0.1,max_depth=3, n_estimators=300)

gb_reg.fit(X, y)

GradientBoostingRegressor(n_estimators=300)

In [39]:
target_2 = target_2[["length [mm]", "chf_exp [MW/m2]"]]
final_2 = gb_reg.predict(target_2)

In [79]:
df_pressure = df.copy()
final_target_2 = df_pressure[df_pressure["pressure [MPa]"].isnull()]
final_target_2 = final_target_2[(final_target_2["length [mm]"].notna())]

final_target_2["pressure [MPa]"] = final_2

In [80]:
final_target_2

,id,length [mm],pressure [MPa],chf_exp [MW/m2]
1,1,762.0,8.485431,6.2
10,10,152.0,15.279082,3.2
14,14,152.0,15.572101,3.8
18,18,432.0,6.717196,3.4
20,20,762.0,7.352114,4.5
...,...,...,...,...
31611,31611,1524.0,10.129523,7.3
31612,31612,457.0,13.778702,2.4
31625,31625,229.0,8.456391,12.8
31638,31638,318.0,13.232652,9.0


# Length & Pressure imputation

In [45]:
df

,id,length [mm],pressure [MPa],chf_exp [MW/m2]
0,0,432.0,7.00,3.6
1,1,762.0,NaN,6.2
2,2,457.0,13.79,2.5
3,3,2134.0,13.79,3.0
4,4,457.0,13.79,2.8
...,...,...,...,...
31639,31639,591.0,NaN,2.3
31640,31640,NaN,13.79,3.9
31641,31641,150.0,18.27,2.3
31642,31642,1972.0,6.89,3.7


In [84]:
df.update(final_target.set_index('id').combine_first(df.set_index('id')))
df.update(final_target_2.set_index('id').combine_first(df.set_index('id')))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31644 entries, 0 to 31643
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               31644 non-null  int64  
 1   length [mm]      31644 non-null  float64
 2   pressure [MPa]   31004 non-null  float64
 3   chf_exp [MW/m2]  31644 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 989.0 KB


In [85]:
df["pressure [MPa]"] = df["pressure [MPa]"].fillna(df["pressure [MPa]"].mean())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31644 entries, 0 to 31643
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               31644 non-null  int64  
 1   length [mm]      31644 non-null  float64
 2   pressure [MPa]   31644 non-null  float64
 3   chf_exp [MW/m2]  31644 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 989.0 KB


In [86]:
df.to_csv("../data/imputed_heat.csv", index=False)